Download overlay and grab DMA aliases

<div class="alert alert-box alert-warning">
For <b>Kria</b> boards use the `Debug Bridge` IP instead of the JTAG cable.
</div>

In [ ]:
from pynq import Overlay, allocate, PL
import numpy as np

PL.reset()

ol = Overlay('dma_subset.bit')
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel

In Vivado open **Open Hardware Manger**, then **Open Target** then **Auto Connect**

Allocate input and output buffer, then initialize input buffer. Output buffer is half the size to trigger a Runtime error

In [ ]:
data_size = 16

input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)

input_buffer[:] = np.arange(data_size, dtype=np.uint32)

In Vivado Hardware Manager, set the trigger to start capturing when TVALID of *axis_data_fifo_M_AXIS* is 1 or in the rising edge.
`RuntimeError: DMA channel not started` will be triggered after a few seconds

In [ ]:
dma_recv.transfer(output_buffer)
dma_send.transfer(input_buffer)
dma_send.wait()
dma_recv.wait()

Check the register map to identify errors

In [ ]:
dma.register_map

Release the buffers and overlay

In [ ]:
del input_buffer, output_buffer
ol.free()